In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os 
import sys

projRootPath =  os.path.abspath(os.path.join('.' ,"../.."))
print("Project root path is: ", projRootPath)
sys.path.append(projRootPath)

from src.utils.path_manipulation import contains_filetype
from src.utils.viz_functions import make_mel_spectrogram
from src.utils.sound_functions import mix_samples, read_mp3, write_mp3, pad_length_to_max

# modules for visuals
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

# modules for data processing
import numpy as np
from scipy import signal
from scipy.io import wavfile
import wave
import struct
import csv
import random
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error 

# modules for data mining
from sklearn.decomposition import pca, FastICA

# and IPython.display for audio output
from tqdm.notebook import tqdm 
import IPython.display

# Librosa for audio
import librosa
# And the display module for visualization
import librosa.display

from pydub import AudioSegment

Project root path is:  D:\Programming\AudioMining


In [3]:
# Get data files
dataDirName = "data"
# rawDataDirName = os.path.join("raw", "mp3")
rawDataDirName = "converted_wav"
className1 = "violin"
className2 = "guitar"
data_path1 = os.path.join(projRootPath, dataDirName, rawDataDirName, className1)
data_path2 = os.path.join(projRootPath, dataDirName, rawDataDirName, className2)

# Get all files from data_path 
# r=root, d=directories, f = files
file_extension = ".wav"
(_, d, allFiles) = next(os.walk(data_path1))
soundFiles1 = [f for f in allFiles if f.endswith(file_extension)]

(_, d, allFiles) = next(os.walk(data_path2))
soundFiles2 = [f for f in allFiles if f.endswith(file_extension)]

### concatenate more notes, duplicate and shuffle, then make model

In [4]:
sdata1 = []
sampleRates1 = []
soundFiles = soundFiles1
for indx, file in tqdm(enumerate(soundFiles)):
    fpath = os.path.join(data_path1, file)
#     soundSample, fs = read_mp3(fpath)
    fs, soundSample = wavfile.read(fpath)
    sdata1.append(soundSample)
    sampleRates1.append(fs)

In [5]:
sdata2 = []
sampleRates2 = []
soundFiles = soundFiles2
for indx, file in tqdm(enumerate(soundFiles)):
    fpath = os.path.join(data_path2, file)
#     soundSample, fs = read_mp3(fpath)
    fs, soundSample = wavfile.read(fpath)
    sdata2.append(soundSample)
    sampleRates2.append(fs)

In [17]:
violinNotes = set([x.split("_")[1] for x in soundFiles1])
print(violinNotes)

{'Fs6', 'B6', 'Fs7', 'Cs4', 'Ds4', 'G7', 'Gs4', 'Gs3', 'F4', 'Cs7', 'Cs5', 'A3', 'Ds7', 'D6', 'G6', 'B7', 'C7', 'As6', 'As4', 'B4', 'C8', 'E6', 'A7', 'D7', 'C4', 'Cs6', 'A5', 'E8', 'As3', 'A4', 'B3', 'F5', 'B5', 'D5', 'Ds5', 'Ds6', 'E4', 'C6', 'D4', 'Gs7', 'Gs5', 'G5', 'Fs5', 'G4', 'F6', 'E5', 'F7', 'Fs4', 'A6', 'C5', 'Gs6', 'As7', 'G3', 'E7', 'As5'}


In [18]:
guitarNotes = set([x.split("_")[1] for x in soundFiles2])
print(guitarNotes)

{'F2', 'Cs4', 'Ds4', 'Gs4', 'G2', 'Gs3', 'F4', 'A3', 'D3', 'Gs2', 'A2', 'As4', 'As2', 'B4', 'E6', 'E3', 'C4', 'C3', 'Cs3', 'As3', 'A4', 'B3', 'B5', 'D5', 'Ds3', 'Ds5', 'E4', 'C6', 'D4', 'Gs5', 'G5', 'Fs5', 'G4', 'Fs3', 'E5', 'B2', 'Fs4', 'E2', 'Fs2', 'C5', 'F3', 'G3'}
